In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel, get_scheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
import pandas as pd
from tqdm import tqdm
import numpy as np


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:

# Config
Baseline_MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
Vanilla_BASELINE_MODEL_NAME = "bert-base-uncased"
MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

# Dataset preparation
class ClinicalNotesDataset(Dataset):
    """
    PyTorch Dataset for tokenizing clinical notes or abstracts.

    Args:
        texts (List[str]): Input text data.
        labels (List[int]): Corresponding labels.
        tokenizer (PreTrainedTokenizer): Tokenizer instance from HuggingFace.
        max_len (int): Maximum sequence length.
    """
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }


## **Read Dataset**

In [ ]:
# Load data
df = pd.read_csv("/content/medical_abstract_data.csv")


# Create a mapping dictionary
condition_mapping = {
    1: 'neoplasms',
    2: 'digestive system diseases',
    3: 'nervous system diseases',
    4: 'cardiovascular diseases',
    5: 'general pathological conditions'
}

# Add a new column 'condition_name' based on 'condition_label'
df['condition_name'] = df['condition_label'].map(condition_mapping)

# Reindex class labels from 1-5 to 0-4
df['condition_label'] = df['condition_label'].astype(int) - 1

# Prepare dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['medical_abstract'].tolist(),
    df['condition_label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['condition_label']
)


In [ ]:

# Training loop
def train_epoch(model, dataloader):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['label'].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)


def evaluate(model, dataloader, save_prefix="evaluation_results", condition_mapping=None):
    """
    Evaluate a model on a dataloader, save predictions, classification report, and AUC/ROC data.

    Args:
        model: torch model
        dataloader: dataloader with batches
        save_prefix: prefix for CSV file names
        condition_mapping: dict {int: str} for class names
    """
    model.eval()
    all_preds, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['label'].to(DEVICE)

            outputs = model(input_ids, attention_mask)  # logits: (batch_size, n_classes)

            probs = torch.softmax(outputs, dim=1)  # probabilities: (batch_size, n_classes)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())  # accumulate probabilities

    # convert lists to arrays
    y_true = np.array(all_labels)
    y_pred = np.array(all_preds)
    y_score = np.array(all_probs)  # shape: (n_samples, n_classes)

    n_classes = y_score.shape[1]
    if condition_mapping is None:
        label_names = [str(i) for i in range(n_classes)]
    else:
        label_names = [condition_mapping[i] for i in range(n_classes)]

    # save predictions & labels
    df_preds = pd.DataFrame({
        'true_label': y_true,
        'predicted_label': y_pred
    })
    preds_csv_path = f"{save_prefix}_predictions.csv"
    df_preds.to_csv(preds_csv_path, index=False)

    # save classification report
    report_dict = classification_report(
        y_true, y_pred,
        target_names=label_names,
        output_dict=True
    )
    report_df = pd.DataFrame(report_dict).transpose().reset_index()
    report_df.rename(columns={"index": "class"}, inplace=True)
    report_csv_path = f"{save_prefix}_report.csv"
    report_df.to_csv(report_csv_path, index=False)

    print(f"✅ Predictions saved: {preds_csv_path}")
    print(f"✅ Classification report saved: {report_csv_path}")
    print(classification_report(y_true, y_pred, target_names=label_names))

    # 🎯 compute AUC and save ROC data
    if n_classes == 2:
        # binary case
        auc = roc_auc_score(y_true, y_score[:, 1])
        fpr, tpr, thresholds = roc_curve(y_true, y_score[:, 1])
        df_roc = pd.DataFrame({
            'fpr': fpr,
            'tpr': tpr,
            'threshold': thresholds
        })
        roc_csv_path = f"{save_prefix}_roc_curve.csv"
        df_roc.to_csv(roc_csv_path, index=False)
        print(f"📄 ROC curve data saved: {roc_csv_path}")
        print(f"🔷 ROC AUC (binary): {auc:.4f}")

    else:
        # multi-class
        y_true_bin = label_binarize(y_true, classes=range(n_classes))
        auc = roc_auc_score(y_true_bin, y_score, multi_class='ovr')
        roc_dfs = []
        for i in range(n_classes):
            fpr, tpr, thresholds = roc_curve(y_true_bin[:, i], y_score[:, i])
            df_roc = pd.DataFrame({
                'fpr': fpr,
                'tpr': tpr,
                'threshold': thresholds
            })
            df_roc['class'] = label_names[i]
            roc_dfs.append(df_roc)

        df_roc_all = pd.concat(roc_dfs, ignore_index=True)
        roc_csv_path = f"{save_prefix}_roc_curve_multiclass.csv"
        df_roc_all.to_csv(roc_csv_path, index=False)
        print(f"📄 ROC curve data (multi-class) saved: {roc_csv_path}")
        print(f"🔷 ROC AUC (multi-class, OVR): {auc:.4f}")


### **Step 1: ClinicalBERT Basic Classifier**

In [ ]:


# Define model
class ClinicalBERTClassifier(nn.Module):
    def __init__(self, base_model, num_classes=5):
        super(ClinicalBERTClassifier, self).__init__()
        self.bert = base_model
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls_output)
        return self.classifier(x)

# Model + Tokenizer Initialization
tokenizer = AutoTokenizer.from_pretrained(Baseline_MODEL_NAME)
bert_model = AutoModel.from_pretrained(Baseline_MODEL_NAME)

#  Dataset Loader
train_dataset = ClinicalNotesDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = ClinicalNotesDataset(val_texts, val_labels, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Model Initialization
model = ClinicalBERTClassifier(bert_model).to(DEVICE)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=EPOCHS * len(train_loader)
)
loss_fn = nn.CrossEntropyLoss()

# Run training and evaluation
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    train_loss = train_epoch(model, train_loader)
    print(f"Train loss: {train_loss:.4f}")
    evaluate(model=model, dataloader=val_loader, save_prefix="BERT_base_classifier")



Epoch 1/5


Training: 100%|██████████| 578/578 [06:22<00:00,  1.51it/s]


Train loss: 0.9464


Evaluating: 100%|██████████| 145/145 [00:30<00:00,  4.77it/s]


✅ Predictions saved: BERT_base_classifier_predictions.csv
✅ Classification report saved: BERT_base_classifier_report.csv
              precision    recall  f1-score   support

           0       0.72      0.84      0.77       506
           1       0.53      0.65      0.59       239
           2       0.58      0.56      0.57       308
           3       0.66      0.83      0.74       488
           4       0.62      0.42      0.50       769

    accuracy                           0.64      2310
   macro avg       0.62      0.66      0.63      2310
weighted avg       0.64      0.64      0.63      2310

📄 ROC curve data (multi-class) saved: BERT_base_classifier_roc_curve_multiclass.csv
🔷 ROC AUC (multi-class, OVR): 0.8987

Epoch 2/5


...

✅ Predictions saved: BERT_base_classifier_predictions.csv
✅ Classification report saved: BERT_base_classifier_report.csv
              precision    recall  f1-score   support

           0       0.69      0.83      0.75       506
           1       0.49      0.59      0.54       239
           2       0.52      0.49      0.50       308
           3       0.66      0.77      0.71       488
           4       0.59      0.42      0.49       769

    accuracy                           0.61      2310
   macro avg       0.59      0.62      0.60      2310
weighted avg       0.61      0.61      0.60      2310

📄 ROC curve data (multi-class) saved: BERT_base_classifier_roc_curve_multiclass.csv
🔷 ROC AUC (multi-class, OVR): 0.8896

Epoch 5/5


Training: 100%|██████████| 578/578 [06:22<00:00,  1.51it/s]


Train loss: 0.5215


Evaluating: 100%|██████████| 145/145 [00:30<00:00,  4.77it/s]

✅ Predictions saved: BERT_base_classifier_predictions.csv
✅ Classification report saved: BERT_base_classifier_report.csv
              precision    recall  f1-score   support

           0       0.69      0.79      0.74       506
           1       0.50      0.56      0.53       239
           2       0.51      0.53      0.52       308
           3       0.66      0.74      0.70       488
           4       0.57      0.43      0.49       769

    accuracy                           0.60      2310
   macro avg       0.58      0.61      0.59      2310
weighted avg       0.60      0.60      0.60      2310

📄 ROC curve data (multi-class) saved: BERT_base_classifier_roc_curve_multiclass.csv
🔷 ROC AUC (multi-class, OVR): 0.8864


### **Step 2 (Vanilla BERT baseline)**

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, base_model, num_classes=5):
        super(BERTClassifier, self).__init__()
        self.bert = base_model
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls_output)
        return self.classifier(x)

# Model + Tokenizer Initialization
baseline_tokenizer = AutoTokenizer.from_pretrained(Vanilla_BASELINE_MODEL_NAME)
baseline_bert = AutoModel.from_pretrained(Vanilla_BASELINE_MODEL_NAME)

#  Dataset Loader
baseline_train_dataset = ClinicalNotesDataset(train_texts, train_labels, baseline_tokenizer, MAX_LEN)
baseline_val_dataset = ClinicalNotesDataset(val_texts, val_labels, baseline_tokenizer, MAX_LEN)
baseline_train_loader = DataLoader(baseline_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
baseline_val_loader = DataLoader(baseline_val_dataset, batch_size=BATCH_SIZE)

# Model Initialization
baseline_model = BERTClassifier(baseline_bert).to(DEVICE)

# Training Loop
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS} [BERT Baseline]")
    train_loss = train_epoch(baseline_model, baseline_train_loader)
    print(f"Train loss: {train_loss:.4f}")
    evaluate(model=baseline_model, dataloader=baseline_val_loader, save_prefix="VanillaBERT_baseline")



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Training: 100%|██████████| 578/578 [06:03<00:00,  1.59it/s]


Train loss: 1.6143


Evaluating: 100%|██████████| 145/145 [00:30<00:00,  4.80it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

✅ Predictions saved: VanillaBERT_baseline_predictions.csv
✅ Classification report saved: VanillaBERT_baseline_report.csv
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       506
           1       0.12      0.14      0.13       239
           2       0.00      0.00      0.00       308
           3       0.24      0.13      0.17       488
           4       0.34      0.76      0.47       769

    accuracy                           0.29      2310
   macro avg       0.14      0.21      0.15      2310
weighted avg       0.17      0.29      0.20      2310

📄 ROC curve data (multi-class) saved: VanillaBERT_baseline_roc_curve_multiclass.csv
🔷 ROC AUC (multi-class, OVR): 0.4948

Epoch 5/5 [BERT Baseline]


Training:   5%|▍         | 27/578 [00:17<05:45,  1.59it/s]

## **Step 3: ClinicalBERT-based Hierarchical Classification**

In [ ]:
# Model Architecture
class HierarchicalClinicalBERT(nn.Module):
    """
    A two-headed ClinicalBERT model for hierarchical classification of medical text.

    Stage 1: Binary classification (General vs. Specific).
    Stage 2: Multiclass classification among Specific classes (0–3).

    Args:
        base_model (nn.Module): Pretrained ClinicalBERT model.

    Forward Pass:
        input_ids (Tensor): Tokenized input text IDs.
        attention_mask (Tensor): Attention masks.

    Returns:
        dict: {
            'stage1_logits': Tensor of shape (batch_size, 2),
            'stage2_logits': Tensor of shape (batch_size, 4)
        }
    """
    def __init__(self, base_model):
        super().__init__()
        self.bert = base_model
        self.dropout = nn.Dropout(0.3)
        self.stage1_head = nn.Linear(self.bert.config.hidden_size, 2)
        self.stage2_head = nn.Linear(self.bert.config.hidden_size, 4)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embed = self.dropout(output.last_hidden_state[:, 0, :])
        return {
            "stage1_logits": self.stage1_head(cls_embed),
            "stage2_logits": self.stage2_head(cls_embed)
        }


#### **Prepare Stage 1 data and loaders**

In [ ]:
# Shared tokenizer
hier_tokenizer = AutoTokenizer.from_pretrained(Baseline_MODEL_NAME)

# Create binary labels (0: Specific, 1: General)
df['binary_label'] = df['condition_label'].apply(lambda x: 1 if x == 4 else 0)

# Split for Stage 1 (all data)
train_texts, val_texts, train_bin, val_bin = train_test_split(
    df['medical_abstract'].tolist(),
    df['binary_label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['binary_label']
)

stage1_train_dataset = ClinicalNotesDataset(train_texts, train_bin, hier_tokenizer, MAX_LEN)
stage1_val_dataset = ClinicalNotesDataset(val_texts, val_bin, hier_tokenizer, MAX_LEN)

stage1_train_loader = DataLoader(stage1_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
stage1_val_loader = DataLoader(stage1_val_dataset, batch_size=BATCH_SIZE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

#### **Prepare Stage 2 data and loaders (multiclass)**

In [ ]:
# Filter only specific classes (0–3)
df_specific = df[df['condition_label'] != 4]

# Split for Stage 2 (specific data)
train_texts_2, val_texts_2, train_labels_2, val_labels_2 = train_test_split(
    df_specific['medical_abstract'].tolist(),
    df_specific['condition_label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df_specific['condition_label']
)

stage2_train_dataset = ClinicalNotesDataset(train_texts_2, train_labels_2, hier_tokenizer, MAX_LEN)
stage2_val_dataset = ClinicalNotesDataset(val_texts_2, val_labels_2, hier_tokenizer, MAX_LEN)

stage2_train_loader = DataLoader(stage2_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
stage2_val_loader = DataLoader(stage2_val_dataset, batch_size=BATCH_SIZE)


### **Stage 1: Binary Classifier — General vs Specific**

#### **Loss, optimizer, scheduler**

In [ ]:
hier_base_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
hier_model = HierarchicalClinicalBERT(hier_base_model).to(DEVICE)

optimizer = AdamW(hier_model.parameters(), lr=LEARNING_RATE)
scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=EPOCHS * len(stage1_train_loader)
)
loss_fn_stage1 = nn.CrossEntropyLoss()


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

#### **Stage1: Training and Evaluation Loop**

In [ ]:
def train_stage(model, loader, logits_key, loss_fn, optimizer, scheduler, desc="Training"):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, desc=desc):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['label'].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[logits_key]
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate_stage(model, loader, logits_key, label_names, desc="Evaluation", save_prefix=None):
    model.eval()
    preds, true, probs = [], [], []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['label'].to(DEVICE)

            logits = model(input_ids=input_ids, attention_mask=attention_mask)[logits_key]
            prob_batch = torch.softmax(logits, dim=1)
            pred_labels = torch.argmax(prob_batch, dim=1)

            preds.extend(pred_labels.cpu().numpy())
            true.extend(labels.cpu().numpy())
            probs.extend(prob_batch.cpu().numpy())



    n_classes = len(label_names)

    # save predictions and labels
    if save_prefix:
        df_preds = pd.DataFrame({
            'true_label': true,
            'predicted_label': preds
        })
        preds_csv_path = f"{save_prefix}_predictions.csv"
        df_preds.to_csv(preds_csv_path, index=False)

        # save classification report as CSV
        report_dict = classification_report(true, preds, target_names=label_names, output_dict=True)
        report_df = pd.DataFrame(report_dict).transpose().reset_index()
        report_df.rename(columns={"index": "class"}, inplace=True)
        report_csv_path = f"{save_prefix}_report.csv"
        report_df.to_csv(report_csv_path, index=False)

        print(f"\n✅ Predictions saved: {preds_csv_path}")
        print(f"✅ Classification report saved: {report_csv_path}")

    # print classification report
    print(f"\n{desc}:")
    print(classification_report(true, preds, target_names=label_names))

    # Compute & save AUC and ROC curve
    print(f'len of probs {probs} and tru labels {true}')
    probs = np.array(probs)  # shape: (n_samples, n_classes)
    y_true = np.array(true)

    if n_classes == 2:
        # binary case
        auc = roc_auc_score(y_true, probs[:, 1])
        fpr, tpr, thresholds = roc_curve(y_true, probs[:, 1])
        df_roc = pd.DataFrame({
            'fpr': fpr,
            'tpr': tpr,
            'threshold': thresholds
        })
        if save_prefix:
            roc_csv_path = f"{save_prefix}_roc_curve.csv"
            df_roc.to_csv(roc_csv_path, index=False)
            print(f"📄 ROC curve data saved: {roc_csv_path}")
        print(f"🔷 ROC AUC (binary): {auc:.4f}")

    else:
        # multi-class
        y_true_bin = label_binarize(y_true, classes=range(n_classes))
        auc = roc_auc_score(y_true_bin, probs, multi_class='ovr')
        roc_dfs = []
        for i in range(n_classes):
            fpr, tpr, thresholds = roc_curve(y_true_bin[:, i], probs[:, i])
            df_roc = pd.DataFrame({
                'fpr': fpr,
                'tpr': tpr,
                'threshold': thresholds
            })
            df_roc['class'] = label_names[i]
            roc_dfs.append(df_roc)

        df_roc_all = pd.concat(roc_dfs, ignore_index=True)
        if save_prefix:
            roc_csv_path = f"{save_prefix}_roc_curve_multiclass.csv"
            df_roc_all.to_csv(roc_csv_path, index=False)
            print(f"📄 ROC curve data (multi-class) saved: {roc_csv_path}")
        print(f"🔷 ROC AUC (multi-class, OVR): {auc:.4f}")


#### **Stage 1 (Binary Classification: Specific vs General):**

In [ ]:

for epoch in range(EPOCHS):
    print(f"\n----- Epoch {epoch+1} / {EPOCHS} -----")

    loss_stage1 = train_stage(
        model=hier_model,
        loader=stage1_train_loader,
        logits_key="stage1_logits",
        loss_fn=loss_fn_stage1,
        optimizer=optimizer,
        scheduler=scheduler,
        desc="Stage 1 Training"
    )

    print(f"Stage 1 Loss: {loss_stage1:.4f}")

    evaluate_stage(
        model=hier_model,
        loader=stage1_val_loader,
        logits_key="stage1_logits",
        label_names=["Specific (0–3)", "General (4)"],
        desc="Stage 1 Evaluation",
        save_prefix="Stage1_Evaluation"
    )



----- Epoch 1 / 5 -----



Stage 1 Training: 100%|██████████| 578/578 [06:32<00:00,  1.47it/s]


Stage 1 Loss: 0.5915

✅ Predictions saved: Stage1_Evaluation_predictions.csv
✅ Classification report saved: Stage1_Evaluation_report.csv

Stage 1 Evaluation:
                precision    recall  f1-score   support

Specific (0–3)       0.73      0.94      0.82      1541
   General (4)       0.73      0.30      0.43       769

      accuracy                           0.73      2310
     macro avg       0.73      0.62      0.62      2310
  weighted avg       0.73      0.73      0.69      2310

len of probs [array([0.77436143, 0.2256386 ], dtype=float32), array([0.8515002 , 0.14849976], dtype=float32), array([0.8408587 , 0.15914138], dtype=float32), array([0.91665304, 0.08334701], dtype=float32), array([0.78511935, 0.21488068], dtype=float32), array([0.6157121 , 0.38428792], dtype=float32), array([0.8292658 , 0.17073418], dtype=float32), array([0.77861744, 0.2213825 ], dtype=float32), array([0.76680297, 0.2331971 ], dtype=float32), array([0.8677558 , 0.13224421], dtype=float32), array([0.

Stage 1 Training: 100%|██████████| 578/578 [06:24<00:00,  1.50it/s]


Stage 1 Loss: 0.5104

✅ Predictions saved: Stage1_Evaluation_predictions.csv
✅ Classification report saved: Stage1_Evaluation_report.csv

Stage 1 Evaluation:
                precision    recall  f1-score   support

Specific (0–3)       0.78      0.81      0.79      1541
   General (4)       0.58      0.54      0.56       769

      accuracy                           0.72      2310
     macro avg       0.68      0.67      0.67      2310
  weighted avg       0.71      0.72      0.71      2310

len of probs [array([0.78650254, 0.21349742], dtype=float32), array([0.77320474, 0.22679529], dtype=float32), array([0.8922181 , 0.10778186], dtype=float32), array([0.9705296 , 0.02947033], dtype=float32), array([0.6984515, 0.3015485], dtype=float32), array([0.29377356, 0.70622647], dtype=float32), array([0.59522504, 0.404775  ], dtype=float32), array([0.40077695, 0.5992231 ], dtype=float32), array([0.7241537 , 0.27584636], dtype=float32), array([0.9106442 , 0.08935582], dtype=float32), array([0.13

...

Stage 1 Loss: 0.4409

✅ Predictions saved: Stage1_Evaluation_predictions.csv
✅ Classification report saved: Stage1_Evaluation_report.csv

Stage 1 Evaluation:
                precision    recall  f1-score   support

Specific (0–3)       0.75      0.84      0.80      1541
   General (4)       0.59      0.45      0.51       769

      accuracy                           0.71      2310
     macro avg       0.67      0.65      0.65      2310
  weighted avg       0.70      0.71      0.70      2310

len of probs [array([0.9027338, 0.0972662], dtype=float32), array([0.81294066, 0.18705937], dtype=float32), array([0.9508013, 0.0491987], dtype=float32), array([0.9905969 , 0.00940316], dtype=float32), array([0.777001  , 0.22299895], dtype=float32), array([0.28951988, 0.71048015], dtype=float32), array([0.53744805, 0.46255195], dtype=float32), array([0.29323083, 0.70676917], dtype=float32), array([0.77734673, 0.22265333], dtype=float32), array([0.9673026 , 0.03269738], dtype=float32), array([0.1734

Stage 1 Training: 100%|██████████| 578/578 [06:25<00:00,  1.50it/s]


Stage 1 Loss: 0.3736

✅ Predictions saved: Stage1_Evaluation_predictions.csv
✅ Classification report saved: Stage1_Evaluation_report.csv

Stage 1 Evaluation:
                precision    recall  f1-score   support

Specific (0–3)       0.76      0.75      0.75      1541
   General (4)       0.51      0.52      0.51       769

      accuracy                           0.67      2310
     macro avg       0.63      0.63      0.63      2310
  weighted avg       0.67      0.67      0.67      2310

len of probs [array([0.9323848 , 0.06761518], dtype=float32), array([0.63379574, 0.36620426], dtype=float32), array([0.9249214, 0.0750786], dtype=float32), array([9.9908257e-01, 9.1738213e-04], dtype=float32), array([0.79770404, 0.2022959 ], dtype=float32), array([0.29999834, 0.7000016 ], dtype=float32), array([0.71568394, 0.28431603], dtype=float32), array([0.18869457, 0.81130546], dtype=float32), array([0.5668267, 0.4331733], dtype=float32), array([0.9791084 , 0.02089165], dtype=float32), array([

Stage 1 Training: 100%|██████████| 578/578 [06:25<00:00,  1.50it/s]


Stage 1 Loss: 0.3197

✅ Predictions saved: Stage1_Evaluation_predictions.csv
✅ Classification report saved: Stage1_Evaluation_report.csv

Stage 1 Evaluation:
                precision    recall  f1-score   support

Specific (0–3)       0.75      0.79      0.77      1541
   General (4)       0.52      0.47      0.49       769

      accuracy                           0.68      2310
     macro avg       0.64      0.63      0.63      2310
  weighted avg       0.67      0.68      0.68      2310

len of probs [array([0.95963895, 0.04036108], dtype=float32), array([0.7745488 , 0.22545116], dtype=float32), array([0.9715262 , 0.02847377], dtype=float32), array([9.9964976e-01, 3.5028346e-04], dtype=float32), array([0.8992727 , 0.10072728], dtype=float32), array([0.2947173, 0.7052827], dtype=float32), array([0.73471534, 0.26528463], dtype=float32), array([0.14070104, 0.85929894], dtype=float32), array([0.63766545, 0.36233455], dtype=float32), array([0.99266136, 0.00733869], dtype=float32), array

### **Stage 2: Multiclass Classifier — Specific Only (0–3)**

#### **New optimizer, loss**

In [ ]:
optimizer_stage2 = AdamW(hier_model.parameters(), lr=LEARNING_RATE)
scheduler_stage2 = get_scheduler(
    "linear", optimizer=optimizer_stage2,
    num_warmup_steps=0,
    num_training_steps=EPOCHS * len(stage2_train_loader)
)
loss_fn_stage2 = nn.CrossEntropyLoss()


#### **Training and Evaluation Loop**

In [ ]:

for epoch in range(EPOCHS):
    print(f"\n----- Epoch {epoch+1} / {EPOCHS} -----")

    loss_stage2 = train_stage(
        model=hier_model,
        loader=stage2_train_loader,
        logits_key="stage2_logits",
        loss_fn=loss_fn_stage2,
        optimizer=optimizer_stage2,
        scheduler=scheduler_stage2,
        desc="Stage 2 Training"
    )

    print(f"Stage 2 Loss: {loss_stage2:.4f}")

    evaluate_stage(
        model=hier_model,
        loader=stage2_val_loader,
        logits_key="stage2_logits",
        label_names=['neoplasms', 'digestive system diseases', 'nervous system diseases', 'cardiovascular diseases'],
        desc="Stage 2 Evaluation",
        save_prefix="Stage2_Evaluation"
    )



----- Epoch 1 / 5 -----


Stage 2 Training: 100%|██████████| 386/386 [04:15<00:00,  1.51it/s]


Stage 2 Loss: 0.4815

✅ Predictions saved: Stage2_Evaluation_predictions.csv
✅ Classification report saved: Stage2_Evaluation_report.csv

Stage 2 Evaluation:
                           precision    recall  f1-score   support

                neoplasms       0.87      0.88      0.87       506
digestive system diseases       0.80      0.81      0.81       239
  nervous system diseases       0.83      0.73      0.78       308
  cardiovascular diseases       0.87      0.93      0.90       489

                 accuracy                           0.85      1542
                macro avg       0.84      0.84      0.84      1542
             weighted avg       0.85      0.85      0.85      1542

len of probs [array([0.9877688 , 0.00341326, 0.00475966, 0.00405835], dtype=float32), array([0.03780265, 0.91687423, 0.0242777 , 0.02104537], dtype=float32), array([0.48292175, 0.48553333, 0.00985143, 0.02169354], dtype=float32), array([0.81445456, 0.0046475 , 0.1635774 , 0.01732061], dtype=float32), a

Stage 2 Training: 100%|██████████| 386/386 [04:16<00:00,  1.51it/s]


Stage 2 Loss: 0.3304

✅ Predictions saved: Stage2_Evaluation_predictions.csv
✅ Classification report saved: Stage2_Evaluation_report.csv

Stage 2 Evaluation:
                           precision    recall  f1-score   support

                neoplasms       0.86      0.86      0.86       506
digestive system diseases       0.76      0.82      0.79       239
  nervous system diseases       0.81      0.75      0.78       308
  cardiovascular diseases       0.89      0.90      0.89       489

                 accuracy                           0.84      1542
                macro avg       0.83      0.83      0.83      1542
             weighted avg       0.84      0.84      0.84      1542

len of probs [array([0.98974025, 0.00325082, 0.0033618 , 0.00364716], dtype=float32), array([0.02190869, 0.9554229 , 0.0148467 , 0.00782172], dtype=float32), array([0.28170213, 0.6963061 , 0.00808828, 0.01390343], dtype=float32), array([0.9001238 , 0.00458881, 0.08238772, 0.01289968], dtype=float32), a

...

Stage 2 Loss: 0.2607

✅ Predictions saved: Stage2_Evaluation_predictions.csv
✅ Classification report saved: Stage2_Evaluation_report.csv

Stage 2 Evaluation:
                           precision    recall  f1-score   support

                neoplasms       0.84      0.85      0.85       506
digestive system diseases       0.76      0.81      0.78       239
  nervous system diseases       0.77      0.70      0.74       308
  cardiovascular diseases       0.87      0.89      0.88       489

                 accuracy                           0.83      1542
                macro avg       0.81      0.81      0.81      1542
             weighted avg       0.83      0.83      0.82      1542

len of probs [array([0.994426  , 0.00143534, 0.00178081, 0.00235791], dtype=float32), array([0.01329624, 0.97567517, 0.00796057, 0.00306804], dtype=float32), array([0.40226126, 0.5833931 , 0.00433848, 0.01000713], dtype=float32), array([0.93492407, 0.00129104, 0.05855464, 0.00523026], dtype=float32), a

Stage 2 Training: 100%|██████████| 386/386 [04:16<00:00,  1.50it/s]


Stage 2 Loss: 0.2146

✅ Predictions saved: Stage2_Evaluation_predictions.csv
✅ Classification report saved: Stage2_Evaluation_report.csv

Stage 2 Evaluation:
                           precision    recall  f1-score   support

                neoplasms       0.82      0.90      0.86       506
digestive system diseases       0.77      0.79      0.78       239
  nervous system diseases       0.80      0.67      0.73       308
  cardiovascular diseases       0.88      0.88      0.88       489

                 accuracy                           0.83      1542
                macro avg       0.82      0.81      0.81      1542
             weighted avg       0.83      0.83      0.83      1542

len of probs [array([9.9617493e-01, 1.1995485e-03, 8.4433751e-04, 1.7811011e-03],
      dtype=float32), array([0.01357253, 0.9792032 , 0.00511028, 0.00211393], dtype=float32), array([0.5820456 , 0.40868932, 0.00202583, 0.00723919], dtype=float32), array([9.9504930e-01, 5.7795382e-04, 2.4303149e-03, 1.9

Stage 2 Training: 100%|██████████| 386/386 [04:16<00:00,  1.50it/s]


Stage 2 Loss: 0.1831

✅ Predictions saved: Stage2_Evaluation_predictions.csv
✅ Classification report saved: Stage2_Evaluation_report.csv

Stage 2 Evaluation:
                           precision    recall  f1-score   support

                neoplasms       0.83      0.87      0.85       506
digestive system diseases       0.78      0.78      0.78       239
  nervous system diseases       0.77      0.71      0.74       308
  cardiovascular diseases       0.88      0.88      0.88       489

                 accuracy                           0.82      1542
                macro avg       0.81      0.81      0.81      1542
             weighted avg       0.82      0.82      0.82      1542

len of probs [array([9.9658775e-01, 8.8196457e-04, 8.2796981e-04, 1.7023893e-03],
      dtype=float32), array([0.01015738, 0.98403937, 0.00421682, 0.0015865 ], dtype=float32), array([0.5377741 , 0.45196572, 0.00187712, 0.00838306], dtype=float32), array([9.9382788e-01, 3.6296988e-04, 4.0721609e-03, 1.7

### **Two-Stage Inference Function**

In [ ]:
def hierarchical_predict(texts, model, tokenizer, max_len=256):
    model.eval()
    predictions = []

    for text in texts:
        encoding = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=max_len,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].to(DEVICE)
        attention_mask = encoding['attention_mask'].to(DEVICE)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            stage1_pred = torch.argmax(outputs['stage1_logits'], dim=1).item()

            if stage1_pred == 1:  # General (4)
                predictions.append(4)
            else:  # Specific class → use stage2_logits
                stage2_pred = torch.argmax(outputs['stage2_logits'], dim=1).item()
                predictions.append(stage2_pred)

    return predictions


#### **Example Usage**

In [ ]:
condition_mapping = {
    0: 'neoplasms',
    1: 'digestive system diseases',
    2: 'nervous system diseases',
    3: 'cardiovascular diseases',
    4: 'general pathological conditions'
}

test_texts = {
    "Tissue changes around loose prostheses. A canine model to investigate the effects of an antiinflammatory agent. The aseptically loosened prosthesis provided a means for investigating the in vivo and in vitro activity of the cells associated with the loosening process in seven dogs. The cells were isolated and maintained in culture for sufficient periods of time so that their biologic activity could be studied as well as the effect of different agents added to the cells in vivo or in vitro. The biologic response as determined by interleukin-1 and prostaglandin E2 activity paralleled the roentgenographic appearance of loosening and the technetium images and observations made at the time of revision surgery. The correlation between clinical, roentgenographic, histologic, and biochemical loosening indicates that the canine model is suitable for investigating the mechanisms of prosthetic failure. A canine model permits the study of possible nonsurgical therapeutic interventions with the ultimate hope of stopping or slowing the loosening process." : "4",
    "Sexually transmitted diseases of the colon, rectum, and anus. The challenge of the nineties. During the past two decades, an explosive growth in both the prevalence and types of sexually transmitted diseases has occurred. Up to 55 percent of homosexual men with anorectal complaints have gonorrhea; 80 percent of the patients with syphilis are homosexuals. Chlamydia is found in 15 percent of asymptomatic homosexual men, and up to one third of homosexuals have active anorectal herpes simplex virus. In addition, a host of parasites, bacterial, viral, and protozoan are all rampant in the homosexual population. Furthermore, the global epidemic of AIDS has produced a plethora of colorectal manifestations. Acute cytomegalovirus ileocolitis is the most common indication for emergency abdominal surgery in the homosexual AIDS population. Along with cryptosporidia and isospora, the patient may present to the colorectal surgeon with bloody diarrhea and weight loss before the diagnosis of human immunodeficiency virus (HIV) disease. Other patients may present with colorectal Kaposi's sarcoma or anorectal lymphoma, and consequently will be found to have seropositivity for HIV. However, in addition to these protean manifestations, one third of patients with AIDS consult the colorectal surgeon with either condylomata acuminata, anorectal sepsis, or proctitis before the diagnosis of HIV disease. Although aggressive anorectal surgery is associated with reasonable surgical results in some asymptomatic HIV positive patients, the same procedures in AIDS (symptomatic HIV positive) patients will often be met with disastrous results. It is incumbent upon the surgeon, therefore, to recognize the manifestations of HIV disease and diagnose these conditions accordingly." : "1",
    "A modified rhombic transposition flap and its application in dermatology. A modified rhombic transposition flap based around a circular primary defect is described. We have found it to be a versatile design that produces good cosmetic results with a low incidence of postoperative complications. It has become the most frequently used transposition flap in our dermatologic practice." : "0"
}

preds = hierarchical_predict(test_texts.keys(), hier_model, hier_tokenizer)
pred_names = [condition_mapping[p] for p in preds]


for text, pred_label in zip(test_texts.keys(), pred_names):
    actual_label = condition_mapping[int(test_texts[text])]
    print(f"Predicted: {pred_label} | Actual: {actual_label}")
    print(f"→ {text[:100]}...\n")



Predicted: general pathological conditions | Actual: general pathological conditions
→ Tissue changes around loose prostheses. A canine model to investigate the effects of an antiinflamma...

Predicted: digestive system diseases | Actual: digestive system diseases
→ Sexually transmitted diseases of the colon, rectum, and anus. The challenge of the nineties. During ...

Predicted: neoplasms | Actual: neoplasms
→ A modified rhombic transposition flap and its application in dermatology. A modified rhombic transpo...

